In this practice code, we are going to use Ozone dataset. Dictionary of this dataset can be found in https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/airquality.html

In [1]:
import pandas as pd
import numpy as np

In [2]:
url = "https://raw.githubusercontent.com/ga-students/SF-DAT-20/master/Data/ozone.csv"
OzoneData = pd.read_csv(url)

#### Explore the dataset and decide which variables suffer from missing data

In [3]:
OzoneData.count()

Ozone      116
Solar.R    146
Wind       153
Temp       153
Month      153
Day        153
dtype: int64

Answer: 

#### Let's drop rows that have missing values in all the columns you indentified above

Hint: in dropna() if you set how = 'all', it will only drop columns that are suffering from missing values at all varaibles you introduce in subset. If you want to get rid of the row that contains missing values in any of the variables you specify, then you shall set how = 'any'

df.dropna(how = 'all',subset = ['Var1','Var2','Var3'],inplace = True)

The above code will check if all 3 variables specified in df have missing values, if they all have missing values it will drop that row.

In [4]:
OzoneData.dropna(how = 'all',subset = ['Solar.R','Ozone'],inplace = True)

In [5]:
OzoneData.count()

Ozone      116
Solar.R    146
Wind       151
Temp       151
Month      151
Day        151
dtype: int64

#### Spoiler! If everything is going according to plan you shall be left by 151 observations. Also, it seemed like the first two varibales had missing values. Now plese make a copy of your dataframe into a dataframe named OzoneImputeMean. Also, please use mean of the variables to fill in missing values in OzoneImputeMean

In [6]:
OzoneImputeMean = OzoneData.copy()
OzoneImputeMean['Ozone'].fillna(value = np.mean(OzoneImputeMean['Ozone']), inplace = True)
OzoneImputeMean['Solar.R'].fillna(value = np.mean(OzoneImputeMean['Solar.R']), inplace = True)

In [7]:
#Check if missing values are filled

OzoneImputeMean.count()

Ozone      151
Solar.R    151
Wind       151
Temp       151
Month      151
Day        151
dtype: int64

#### Now it's time for imputing using linear regression lines

In [8]:
#?????
url = "https://raw.githubusercontent.com/ga-students/SF-DAT-20/master/Data/ozone.csv"
OzoneData = pd.read_csv(url)#### Before we start let's define dummy variables for variable Month - don't worry about day!
#OzoneData.dropna(how = 'all',subset = ['Solar.R','Ozone'],inplace = True)
#??????
MonthDummy = pd.get_dummies(OzoneData.Month, prefix = 'Month')
del MonthDummy['Month_9']
OzoneData = pd.concat([OzoneData, MonthDummy], axis=1)
OzoneData.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
0,41,190,7.4,67,5,1,1,0,0,0
1,36,118,8.0,72,5,2,1,0,0,0
2,12,149,12.6,74,5,3,1,0,0,0
3,18,313,11.5,62,5,4,1,0,0,0
4,NaN,NaN,14.3,56,5,5,1,0,0,0


In [9]:
OzoneData.corr()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
Ozone,1.000000,0.348342,-0.601547,0.698360,0.164519,-0.013226,-0.302963,-0.112006,0.277961,0.291808
Solar.R,0.348342,1.000000,-0.056792,0.275840,-0.075301,-0.150275,-0.024601,0.023998,0.176744,-0.076390
Wind,-0.601547,-0.056792,1.000000,-0.457988,-0.178293,0.027181,0.239025,0.043480,-0.145790,-0.167091
Temp,0.698360,0.275840,-0.457988,1.000000,0.420947,-0.130593,-0.659014,0.063741,0.321700,0.325147
Month,0.164519,-0.075301,-0.178293,0.420947,1.000000,-0.007962,-0.711721,-0.347505,0.002334,0.359361
Day,-0.013226,-0.150275,0.027181,-0.130593,-0.007962,1.000000,0.011187,-0.016988,0.011187,0.011187
Month_5,-0.302963,-0.024601,0.239025,-0.659014,-0.711721,0.011187,1.000000,-0.248948,-0.254098,-0.254098
Month_6,-0.112006,0.023998,0.043480,0.063741,-0.347505,-0.016988,-0.248948,1.000000,-0.248948,-0.248948
Month_7,0.277961,0.176744,-0.145790,0.321700,0.002334,0.011187,-0.254098,-0.248948,1.000000,-0.254098
Month_8,0.291808,-0.076390,-0.167091,0.325147,0.359361,0.011187,-0.254098,-0.248948,-0.254098,1.000000


#### What seems to be the list of best variables can define Ozone? how about Solar.R?

Answer: Temp, Wind, Solar.R, Month_5 ...  ; Ozone, Temp

In [10]:
# now let's use a regression model to predict Ozone. First drop NaN values in Ozone and save it in OzoneDroppedValues_Ozone
# the run a regression line on variables of interest and check significancy of your model
# if in a multi class dummy variable case, you see a few of the dummy variables are not significant but the rest are, 
#you shall either drop all or keep all. Otherwise, selecting the base dummy will become important
# use these variables ['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8'] to predict
from sklearn.linear_model import LinearRegression
from sklearn import feature_selection
OzoneDroppedValues_Ozone=OzoneData.dropna(subset = ['Ozone','Solar.R'])
print OzoneDroppedValues_Ozone.count()
linreg = LinearRegression()

X = OzoneDroppedValues_Ozone[['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8']]
y = OzoneDroppedValues_Ozone['Ozone']
linreg.fit(X,y)
pvals = feature_selection.f_regression(X,y)[1] #That's how we extract p-values
print(pvals)

Ozone      111
Solar.R    111
Wind       111
Temp       111
Month      111
Day        111
Month_5    111
Month_6    111
Month_7    111
Month_8    111
dtype: int64
[  1.79310857e-04   9.08941533e-13   1.55267723e-17   2.43194219e-03
   2.35681480e-01   2.51228945e-03   3.33469888e-03]


In [11]:
# now fill in null values of OzoneData['Ozone'] by predicted values
X = OzoneData[['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8']]
m=np.mean(X['Solar.R'])
X['Solar.R'].fillna(value = m, inplace = True)
print X.describe()
OzoneData['OZ_predict'] = linreg.predict(X)

OzoneData['Ozone'].fillna(value = OzoneData['OZ_predict'], inplace = True)
OzoneData.count()



          Solar.R        Wind        Temp     Month_5     Month_6     Month_7  \
count  153.000000  153.000000  153.000000  153.000000  153.000000  153.000000   
mean   185.931507    9.957516   77.882353    0.202614    0.196078    0.202614   
std     87.960267    3.523001    9.465270    0.403268    0.398332    0.403268   
min      7.000000    1.700000   56.000000    0.000000    0.000000    0.000000   
25%    120.000000    7.400000   72.000000    0.000000    0.000000    0.000000   
50%    194.000000    9.700000   79.000000    0.000000    0.000000    0.000000   
75%    256.000000   11.500000   85.000000    0.000000    0.000000    0.000000   
max    334.000000   20.700000   97.000000    1.000000    1.000000    1.000000   

          Month_8  
count  153.000000  
mean     0.202614  
std      0.403268  
min      0.000000  
25%      0.000000  
50%      0.000000  
75%      0.000000  
max      1.000000  


/Users/karla/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2862: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


Ozone         153
Solar.R       146
Wind          153
Temp          153
Month         153
Day           153
Month_5       153
Month_6       153
Month_7       153
Month_8       153
OZ_predict    153
dtype: int64

In [12]:
# Now repeat previous steps for Solar.R variable using ['Ozone','Wind','Temp]
DroppedValues=OzoneData.dropna(subset = ['Solar.R'])
print DroppedValues.count()
linreg = LinearRegression()

X = DroppedValues[['Ozone','Temp']]
y = DroppedValues['Solar.R']
linreg.fit(X,y)

#Check p-values of your model - if a variable is not significant, drop it
pvals = feature_selection.f_regression(X,y)[1] #That's how we extract p-values
print(pvals)
### We shall drop Wind and rerun our regressions again


Ozone         146
Solar.R       146
Wind          146
Temp          146
Month         146
Day           146
Month_5       146
Month_6       146
Month_7       146
Month_8       146
OZ_predict    146
dtype: int64
[  4.79793585e-05   7.51772924e-04]


In [13]:
# now fill in null values of OzoneData['Solar.R'] by predicted values
X = OzoneData[['Ozone','Temp']]
OzoneData['SR_predict'] = linreg.predict(X)

OzoneData['Solar.R'].fillna(value = OzoneData['SR_predict'], inplace = True)
OzoneData.count()

Ozone         153
Solar.R       153
Wind          153
Temp          153
Month         153
Day           153
Month_5       153
Month_6       153
Month_7       153
Month_8       153
OZ_predict    153
SR_predict    153
dtype: int64

#### Now check your filled data - if your predicted values are more than maximum or less than minimum, replace them by max and min

In [14]:
OZ = [OzoneDroppedValues_Ozone['Ozone'].min(),OzoneDroppedValues_Ozone['Ozone'].max()]
SR = [DroppedValues['Solar.R'].min(),DroppedValues['Solar.R'].max()]

OzoneData.reset_index(drop=True)

for i in range(0,len(OzoneData)):
    if (OzoneData.loc[i,'Ozone'] < OZ[0]):
        OzoneData.loc[i,'Ozone'] = OZ[0]
    if (OzoneData.loc[i,'Solar.R'] < SR[0]):
        OzoneData.loc[i,'Solar.R'] = SR[0]
print(OzoneData['Ozone'].min())


# we don't want to predict above max
for i in range(0,len(OzoneData)):
    if (OzoneData.loc[i,'Ozone'] > OZ[1]):
        OzoneData.loc[i,'Ozone'] = OZ[1]
    if (OzoneData.loc[i,'Solar.R'] > SR[1]):
        OzoneData.loc[i,'Solar.R'] = SR[1]
print(OzoneData['Ozone'].max())
    


1.0
168.0


In [15]:
print(OzoneData['Solar.R'].max())
print(OzoneData['Solar.R'].min())


334.0
7.0


## Bonus: In original question, replace missing values of Ozone and Solar.R by Single Imputation Regression Line with Error

In [16]:
url = "https://raw.githubusercontent.com/ga-students/SF-DAT-20/master/Data/ozone.csv"
OzoneData = pd.read_csv(url)
OzoneData.dropna(how = 'all', subset = ['Ozone', 'Solar.R'], inplace = True)





